# Real data

In [ ]:
import os; from os import listdir; from os.path import isfile, join
import re  
#from skimage import io 
from skimage.io import imread
# To manipulate arrays
import numpy as np 
import random
from tqdm.notebook import tqdm
from timeit import default_timer as timer
import scipy
import pandas as pd
import shutil
import pathlib
import sys
import rsnapsim as rss

from skimage.exposure import rescale_intensity
# Plotting
import matplotlib as mpl ; import matplotlib.pyplot as plt 
# To create interactive elements
import ipywidgets as widgets ; from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout ; from IPython.display import Image
import dna_features_viewer
from dna_features_viewer import BiopythonTranslator, GraphicFeature, GraphicRecord, CircularGraphicRecord

# Deffining directories
current_dir = pathlib.Path().absolute()
sequences_dir = current_dir.parents[1].joinpath('DataBases','gene_files')
video_dir = current_dir.parents[1].joinpath('DataBases','LargeDataSets','20190909_u2os_multiplex','smFLAG-KDM5B')
#video_dir = current_dir.parents[1].joinpath('DataBases','LargeDataSets','20190909_u2os_multiplex','smFLAG-ActB')

rsnaped_dir = current_dir.parents[1].joinpath('rsnaped')

# Importing rSNAPed
sys.path.append(str(rsnaped_dir))
import rsnaped as rsp


In [ ]:
# Paths to dropbox
from sys import platform as _platform
if _platform == "linux" or _platform == "linux2":
    dropbox_address = pathlib.Path('/','home','luisub','Dropbox', 'Project_rSNAPed','manuscript','Figures')
elif _platform == "darwin":
    dropbox_address = pathlib.Path('/','Users','luisaguilera','Dropbox', 'Project_rSNAPed','manuscript','Figures')

In [ ]:
#Function to detect all tif files in a folder
def folder_reader(folder):
    '''
    This function imports all tif files in a give folder.
    Input:
    folder: str, with the file name.

    Returns:
    list_files_names :  List of str with all file names
    path_files : List of str with all file paths
    '''
    # Reads the folder with the results and import the simulations as lists
    list_files_names = sorted([f for f in listdir(folder) if isfile(join(folder, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
    list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
    path_files = [ str(folder.joinpath(f).resolve()) for f in list_files_names ] # creating the complete path for each file
    # Reading the microscopy data
    max_frames_to_read = 100
    list_videos = [imread(f)[0:max_frames_to_read,:,:,:] for f in  path_files] # List with all the videos. 
    num_videos = len(path_files) # the number of videos in the folder is the length of list_videos
    return path_files, list_files_names,list_videos, num_videos

In [ ]:
# Load gene sequences
path_files,list_files_names,list_videos, num_videos = folder_reader(video_dir)
print(path_files)

In [ ]:
# Coding sequence
#gene_file = str(sequences_dir.joinpath('Bactin_withTags.txt')) # coding sequence for SM_KDM5B_PP7    ### 5685 nt   ### 1895 codons
gene_file= str(sequences_dir.joinpath('KDM5B_withTags.txt'))  # coding sequence for SM_p300_MS2      ### 8268 nt   ### 2756 codons

In [ ]:
# Simulating a TASEP model for the beta-actin gene
poi_strs, poi_objs, tagged_pois,raw_seq = rss.seqmanip.open_seq_file(gene_file)  #open this file and find all valid protein of interest objs
gene_mrna = tagged_pois['1'][0]  #lets take the first one since thats the longest
rss.solver.protein = gene_mrna #pass this protein object to the solver
gene_mrna.visualize_probe()  # what does our construct look like?

In [ ]:
intensity_calculation_method = 'disk_donut'  # options are : 'total_intensity' and 'disk_donut' 'gaussian_fit'
mask_selection_method = 'max_area' # options are : 'max_spots' and 'max_area' 
use_optimization_for_tracking = 0 # 0 not using, 1 is using optimization
#frame_selection_empty_video = 'constant' # Options are: 'constant' , 'shuffle' and 'loop'
#particle_size = 3 # spot size for the simulation and tracking.
particle_detection_size = 5 # spot size for the simulation and tracking.

In [ ]:
def plot_original_images(image,slice=0):
    '''
    This function plots all the channels for the original image.
    '''
    number_channels = image.shape[3]
    fig, axes = plt.subplots(nrows=1, ncols=number_channels, figsize=(15, 5))
    for i in range (0,number_channels ):
        img_2d = image[slice,:,:,i]
        img_2d_rescaled = rsp.RemoveExtrema(img_2d, min_percentile = 0.5, max_percentile= 99.5, selected_channels=[0,1]).remove_outliers()
        axes[i].imshow(img_2d_rescaled, cmap='viridis') 
        axes[i].set_title('Channel_'+str(i))
    plt.show()

In [ ]:
plot_original_images(list_videos[0])
plot_original_images(list_videos[1])

In [ ]:
list_DataFrame_particles_intensities= []
for i, temp_video in enumerate(list_videos):
    DataFrame_particles_intensities, _, _, _,_, _, _ = rsp.PipelineTracking(temp_video,particle_size=particle_detection_size,file_name=list_files_names[i],selected_channel=0,intensity_calculation_method =intensity_calculation_method, mask_selection_method = mask_selection_method,show_plot=1,use_optimization_for_tracking=use_optimization_for_tracking,real_positions_dataframe = None, average_cell_diameter=200, print_process_times=1).run()    
    list_DataFrame_particles_intensities.append(DataFrame_particles_intensities)

In [ ]:
all_data = True
sel_timepoint = 0
sel_cell = 0

In [ ]:
df =list_DataFrame_particles_intensities[sel_cell]
df

In [ ]:
print(list_videos[sel_cell][0,:,:,0].mean())
print(list_videos[sel_cell][0,:,:,1].mean())
print(list_videos[sel_cell][0,:,:,2].mean())

# Plotting figures

In [ ]:
fig_size = (20, 5)
f = plt.figure(figsize=fig_size)
def fun_scatter_plot(x,y, title='', xlab='', ylab='',
                alpha=1, color='r',
                subplots=False, show_grid=True, fig=f,font_size=8):
    m, b = np.polyfit(x,y, 1)
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    #axarr = f.add_subplot(1,1,1)
    plt.scatter(x, y, alpha=alpha, color=color, s= 2)
    plt.plot(x, m*x + b,'-w',linewidth=1)
    plt.plot(mean_x,mean_y,'o',color='w' ,  markersize=10)
    plt.xlim(min(x), max(x))
    plt.xlabel(xlab, size=font_size)
    plt.ylabel(ylab, size=font_size)
    plt.grid(show_grid)
    plt.title(title, size=font_size)
    return(f)

## Channels

In [ ]:
selected_timePoint = 0 # selecting a frame to display in the plot.
fig_size = (20, 10)
fig, ax = plt.subplots(ncols=2,nrows=1, figsize=fig_size)
for i in range(0,2):
  img = list_videos[sel_cell][selected_timePoint,:,:,i]
  mean_int = img.mean()
  #img= rsp.RemoveExtrema(img,min_percentile=1,max_percentile=99.9).remove_outliers()
  ax[i].imshow(img,cmap='Greys_r')
  ax[i].axis('off')
  ax[i].set_title(' --REAL-- Channel_'+str(i) + ' Mean_Int = ' +str(round(mean_int,0)) , size=16)
plt.savefig("./results/real_cell.png")
plt.show()


## Intensity distribution

In [ ]:
plt.style.use('ggplot')

In [ ]:
fig_size = (14, 2.5)
font_size = 8 



In [ ]:
f = plt.figure(figsize=fig_size)

channel = 0
f.add_subplot(1,5,1) 
mean_int = list_videos[0][selected_timePoint,:,:,channel].mean()
plt.imshow(list_videos[0][selected_timePoint,:,:,channel],cmap='Greys_r')
plt.axis('off')
#plt.title(' REAL CH_'+str(channel) + ' mu_Int = ' +str(round(mean_int,0)) , size=16)

f.add_subplot(1,5,2) 
fun_scatter_plot(x = df.red_int_mean.values, y= df.SNR_red.values, title='Int vs SNR', xlab='Int', ylab='SNR', fig=f, color='r')
f.add_subplot(1,5,3) 
fun_scatter_plot(x = df.background_int_mean_red.values, y= df.SNR_red.values, title='BG_Int vs SNR', xlab='BG_Int', ylab='SNR', fig=f, color='r')
f.add_subplot(1,5,4) 
fun_scatter_plot(x = df.background_int_std_red.values, y= df.SNR_red.values, title='BG_std_Int vs SNR', xlab='BG_std_Int', ylab='SNR', fig=f, color='r')

# adding the histogram
f.add_subplot(1,5,5) # here is where you add the subplot to f
f=plt.hist(df.SNR_red.values,bins=30,color='r')
#plt.title('dist SNR', size=16)
plt.ylabel('count', size=16)
plt.xlabel('SNR', size=16)

plt.tight_layout() 
plt.subplots_adjust(wspace=0.25, hspace=0)

name_figure = 'real_SNR_ch0.png'
figure_directory = str(dropbox_address.joinpath(name_figure ) )
plt.savefig(figure_directory, transparent=False,dpi=300, bbox_inches = "tight")

plt.show()

In [ ]:
f = plt.figure(figsize=fig_size)

channel = 1
f.add_subplot(1,5,1) 
mean_int = list_videos[0][selected_timePoint,:,:,channel].mean()
plt.imshow(list_videos[0][selected_timePoint,:,:,channel],cmap='Greys_r')
plt.axis('off')
#plt.title(' REAL CH_'+str(channel) + ' mu_Int = ' +str(round(mean_int,0)) , size=16)

f.add_subplot(1,5,2) 
fun_scatter_plot(x = df.green_int_mean.values, y= df.SNR_green.values, title='Int vs SNR', xlab='Int', ylab='SNR', fig=f, color='g',font_size=font_size)
f.add_subplot(1,5,3) 
fun_scatter_plot(x = df.background_int_mean_green.values, y= df.SNR_green.values, title='BG_Int vs SNR', xlab='BG_Int', ylab='SNR', fig=f, color='g',font_size=font_size)
f.add_subplot(1,5,4) 
fun_scatter_plot(x = df.background_int_std_green.values, y= df.SNR_green.values, title='BG_std_Int vs SNR', xlab='BG_std_Int', ylab='SNR', fig=f, color='g',font_size=font_size)

# adding the histogram
f.add_subplot(1,5,5) # here is where you add the subplot to f
f=plt.hist(df.SNR_green.values,bins=30,color='g')
#plt.title('dist SNR', size=16)
plt.ylabel('count', size=font_size)
plt.xlabel('SNR', size=font_size)

plt.tight_layout() 
plt.subplots_adjust(wspace=0.25, hspace=0)

name_figure = 'real_SNR_ch1.png'
figure_directory = str(dropbox_address.joinpath(name_figure ) )
plt.savefig(figure_directory, transparent=True,dpi=300, bbox_inches = "tight")

plt.show()
